In [2]:
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами
import matplotlib.pyplot as plt  # Пакет для строительства графиков
import seaborn as sns            # Еще один пакет для графиков
import pickle                    # Пакет для подгрузки данных специфического для питона формата или для сохранения данных в файл

from wordcloud import WordCloud  # Пакет для построения облаков слов

plt.style.use('ggplot')          # задаём красивый стиль для графиков matplotlib-а
%matplotlib inline

In [3]:
# plt.rcParams['figure.figsize']=15,15
# topic_sim=[]
# for i in range(20):
#   v1 = ' '.join(topic_base[topic_base['topic']==i]['word_in topic'].values[:15])
#   v1_emb=embed_bert_cls(v1, model, tokenizer)
# #  print(v1)
#   for m in range(20):
#     sim_work=[]
#     v2 = ' '.join(topic_nmf[topic_nmf['topic']==m]['topic_word'].values[:15])
#     v2_emb=embed_bert_cls(v2, model, tokenizer)
#     sim=similarity(v1_emb, v2_emb)
#     sim_work.append(i)
#     sim_work.append(m)
#     sim_work.append(sim)
#     topic_sim.append(sim_work)
# #topic_sim
# topic_sim_ds=pd.DataFrame(topic_sim,columns=['base','nmf','cos_sim'])
# glue = topic_sim_ds.pivot(index='base', columns='nmf', values='cos_sim')
# sns.heatmap(glue, annot=True, fmt=".1f")

In [4]:
import sys
!{sys.executable} -m pip install --upgrade pymorphy2

from pymorphy2 import MorphAnalyzer

parser = MorphAnalyzer()

In [5]:
import requests
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer

# Загрузка файла
url = "https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt"
response = requests.get(url)
rus_stops = response.text.splitlines()

punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'
filter_token = rus_stops + list(punctuation)

parser = MorphAnalyzer()

def preprocess_gemsim(input_text):
    text = input_text.lower()
    tokenized_text = word_tokenize(text)
    clean_text = [word for word in tokenized_text if word not in filter_token]
    lemmatized_text = [parser.parse(word)[0].normal_form for word in clean_text if word]  # Проверка на пустые строки
    return lemmatized_text


<>:10: SyntaxWarning: invalid escape sequence '\]'
<>:10: SyntaxWarning: invalid escape sequence '\]'
C:\Users\Сократ\AppData\Local\Temp\ipykernel_20780\1493062947.py:10: SyntaxWarning: invalid escape sequence '\]'
  punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'


In [6]:
import pandas as pd

# Открытие файла с явным указанием кодировки
with open("../anek_utf8.txt", "r", encoding="utf-8") as file1:
    lines = []
    for line in file1:
        if line.strip():  # Проверка на пустую строку
            lines.append(line.strip())  # Удаление пробелов и символов новой строки

line_new = []
for i in range(len(lines) - 1):  # Используем 'i' вместо 'n'
    if lines[i][:15] == '<|startoftext|>':
        if lines[i + 1][:15] != '<|startoftext|>':
            line_new.append(lines[i][15:] + ' ' + lines[i + 1][15:])
        else:
            line_new.append(lines[i][15:])

# Создание DataFrame
ds_line = pd.DataFrame(line_new, columns=['text_base'])


In [7]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')#для табов

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Сократ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Сократ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [8]:
import nltk
nltk.download('punkt')

# ds_line["text_processed_new"] = ds_line["text_base"].apply(lambda text: preprocess_sklearn(text))
ds_line["text_processed"] = ds_line["text_base"].apply(lambda text: preprocess_gemsim(text))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Сократ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
# Build the bigram  models
texts=(ds_line["text_processed"])
from gensim import corpora, models

bigram = models.Phrases(texts, min_count=3, threshold=5)
bigram_mod = models.phrases.Phraser(bigram)
texts_bi = make_bigrams(texts)
dictionary_bi = corpora.Dictionary(texts_bi)                 # составляем словарь из терминов с учетом биграмм слов предлжений
print('Размер словаря до фильтрации: {}'.format(len(dictionary_bi)))
dictionary_bi.filter_extremes(no_below=3, no_above=0.4, keep_n=3*10**6)
print('Размер словаря после фильтрации: {}'.format(len(dictionary_bi)))
corpus_bi = [dictionary_bi.doc2bow(text) for text in texts_bi]  # составляем корпус документов по словарю полученному с использованием биграмм

Размер словаря до фильтрации: 83955
Размер словаря после фильтрации: 37948


In [10]:
number_topics=20

In [11]:
def lda_model_gensim(corpus,dictionary,num_topics):
  ldamodel = models.ldamodel.LdaModel(
    corpus,
    id2word=dictionary,
    eval_every=20,
    num_topics=num_topics,
    passes=5
    )
  return ldamodel

In [12]:
ldamodel_bi=lda_model_gensim(corpus_bi,dictionary_bi,number_topics)

In [13]:
topics_gensim_bi = ldamodel_bi.show_topics(num_topics=number_topics, num_words=100,formatted=False)
topics_bi=pd.DataFrame(topics_gensim_bi)
topics_bi.columns=['topic num','topic_word']

In [14]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [15]:
vis_bi = gensimvis.prepare(ldamodel_bi, corpus_bi, dictionary_bi)

In [16]:
pyLDAvis.enable_notebook()

In [17]:
vis_bi

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
14     0.145263  0.006622       1        1  6.994004
1     -0.239297  0.317003       2        1  6.196069
17     0.131158  0.082649       3        1  5.795503
13     0.121387  0.062595       4        1  5.313608
12     0.034288 -0.013107       5        1  5.221702
10     0.008047 -0.091327       6        1  5.186889
9      0.091775  0.065874       7        1  5.102133
4     -0.183783  0.064089       8        1  5.087674
6      0.128423 -0.023682       9        1  5.081282
19     0.045204 -0.029488      10        1  5.063039
18     0.042341  0.045683      11        1  4.998943
8      0.149898  0.030258      12        1  4.871071
3     -0.061434 -0.101920      13        1  4.732985
7      0.020932  0.063536      14        1  4.671832
15    -0.026406 -0.014941      15        1  4.517178
16    -0.046511 -0.004887      16        1  4.472711
2     -0.085021 -0.054756      17        1  4.470374
0     -0.043339 -0.048431      18        1  4.299273
11    -0.190802 -0.210693      19        1  3.991716
5     -0.042124 -0.145076      20        1  3.932014, topic_info=            Term          Freq         Total Category  logprob  loglift
24            ``  19160.000000  19160.000000  Default  30.0000  30.0000
23            ''  17433.000000  17433.000000  Default  29.0000  29.0000
320         жена   9091.000000   9091.000000  Default  28.0000  28.0000
27       женщина   7480.000000   7480.000000  Default  27.0000  27.0000
849      мужчина   5624.000000   5624.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
5501     граница    237.775551    239.064045  Topic20  -5.1607   3.2306
1036   поступить    233.630888    234.919382  Topic20  -5.1782   3.2305
74           ...    841.353959  39935.370710  Topic20  -3.8970  -0.6240
637   спрашивать    302.350675   4161.996461  Topic20  -4.9204   0.6139
664       сидеть    273.860853   2227.876497  Topic20  -5.0194   1.1398

[737 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
23        2  0.999943        ''
2441     17  0.997521     ''_.-
74        1  0.105821       ...
74        2  0.005559       ...
74        3  0.092024       ...
...     ...       ...       ...
777       5  0.998273     школа
1079     18  0.995890   экзамен
3909      3  0.995545  являться
1055     18  0.997969      язык
1640      8  0.997395      яйцо

[854 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 2, 18, 14, 13, 11, 10, 5, 7, 20, 19, 9, 4, 8, 16, 17, 3, 1, 12, 6])

In [18]:
import requests
from pymorphy2 import MorphAnalyzer

# Загрузка файла со стоп-словами
url = "https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt"
response = requests.get(url)

# Проверка успешности загрузки
if response.status_code == 200:
    rus_stops = response.text.splitlines()  # Разделяем текст на строки
else:
    print("Ошибка при загрузке файла:", response.status_code)

# Определение знаков препинания
punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—»«...–'
filter_token = rus_stops + list(punctuation)
stop_words=filter_token
# Инициализация морфологического анализатора
morph = MorphAnalyzer()

# Теперь вы можете использовать rus_stops и filter_token в вашем коде


In [19]:
import requests

# Загрузка файла со стоп-словами
url = "https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt"
response = requests.get(url)

# Проверка успешности загрузки
if response.status_code == 200:
    rus_stops = response.text.splitlines()  # Разделяем текст на строки
else:
    print("Ошибка при загрузке файла:", response.status_code)

# Определение знаков препинания
punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—»«...–'
filter_token = rus_stops + list(punctuation)
filter = filter_token

# Теперь filter содержит список стоп-слов и знаков препинания


In [20]:
# Преобразуем каждый список в строку
ds_line["text_processed"] = ds_line["text_processed"].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Теперь можно снова выполнить подгонку векторизатора
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=filter)
tfidf = tfidf_vectorizer.fit_transform(ds_line["text_processed"].values.astype('U'))



tokens_sklearn = tfidf_vectorizer.get_feature_names_out()
from sklearn.feature_extraction.text import TfidfVectorizer#  TF-IDF (Term Frequency-Inverse Document Frequency).

no_topics=20
from sklearn.decomposition import NMF, LatentDirichletAllocation# Non-negative Matrix Factorization и Latent Dirichlet Allocation

nmf = NMF(n_components=no_topics, random_state=1024, l1_ratio=.5, init='nndsvd').fit(tfidf)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

# Преобразуем каждый список в строку
ds_line["text_processed"] = ds_line["text_processed"].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Создаём TF-IDF матрицу
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=filter_token)
tfidf = tfidf_vectorizer.fit_transform(ds_line["text_processed"].values.astype('U'))

# Применяем NMF
no_topics = 20
nmf_model = NMF(n_components=no_topics, random_state=1024, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Получение топ-слов для каждой темы
nmf_top_words = []
feature_names = tfidf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(nmf_model.components_):
    nmf_top_words.append([feature_names[i] for i in topic.argsort()[:-16:-1]])  # Топ-15 слов


In [22]:
# plt.rcParams['figure.figsize']=15,15
# topic_sim=[]
# for i in range(20):
#   v1 = ' '.join(topic_base[topic_base['topic']==i]['word_in topic'].values[:15])
#   v1_emb=embed_bert_cls(v1, model, tokenizer)
# #  print(v1)
#   for m in range(20):
#     sim_work=[]
#     v2 = ' '.join(topic_nmf[topic_nmf['topic']==m]['topic_word'].values[:15])
#     v2_emb=embed_bert_cls(v2, model, tokenizer)
#     sim=similarity(v1_emb, v2_emb)
#     sim_work.append(i)
#     sim_work.append(m)
#     sim_work.append(sim)
#     topic_sim.append(sim_work)
# #topic_sim
# topic_sim_ds=pd.DataFrame(topic_sim,columns=['base','nmf','cos_sim'])
# glue = topic_sim_ds.pivot(index='base', columns='nmf', values='cos_sim')
# sns.heatmap(glue, annot=True, fmt=".1f")

In [23]:
from gensim import corpora, models

# Корпус для LDA
texts = ds_line["text_processed"].apply(lambda x: x.split())  # Токенизация текстов
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Применяем LDA
lda_model = models.LdaModel(corpus, num_topics=no_topics, id2word=dictionary, passes=5, random_state=1024)

# Получение топ-слов для каждой темы
lda_top_words = []
for topic_id in range(no_topics):
    lda_top_words.append([word for word, prob in lda_model.show_topic(topic_id, topn=15)])  # Топ-15 слов


In [31]:
print(torch.cuda.is_available())


False


In [32]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
# model.cuda()  # uncomment it if you have a GPU

ImportError: 
AutoModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [25]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=False, return_tensors='pt')

    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [24]:
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import seaborn as sns
import pandas as pd

# Инициализация модели и токенизатора
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Функция для получения эмбеддингов
def embed_bert_cls(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy().flatten()

# Вычисление косинусного сходства между темами
topic_sim = []
for i, v1_words in enumerate(lda_top_words):
    v1 = ' '.join(v1_words)
    v1_emb = embed_bert_cls(v1, model, tokenizer)
    for j, v2_words in enumerate(nmf_top_words):
        v2 = ' '.join(v2_words)
        v2_emb = embed_bert_cls(v2, model, tokenizer)
        sim = cosine_similarity([v1_emb], [v2_emb])[0][0]
        topic_sim.append([i, j, sim])

# Построение тепловой карты
topic_sim_df = pd.DataFrame(topic_sim, columns=['LDA', 'NMF', 'Cosine_Similarity'])
heatmap_data = topic_sim_df.pivot(index='LDA', columns='NMF', values='Cosine_Similarity')
sns.heatmap(heatmap_data, annot=True, fmt=".2f", cmap="coolwarm")


c:\Users\Сократ\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
c:\Users\Сократ\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Сократ\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more deta

ImportError: 
AutoModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
